In [2]:
import os
import pandas as pd
MODEL_NAME = 'Clinical-LSTM'
from helpers import convert_to_nested_list, convert_to_list, explode_train_target
import torch
import numpy as np


In [3]:


model_out_dir = f'{os.getenv("BHF_ROOT")}/model_outputs/{MODEL_NAME}_rad_all_out'
train_emb_seq_path = f'{model_out_dir}/train_emb_seq.csv' # tmp csv for multiple runs
train_target_path = f'{os.getenv("BHF_ROOT")}/JSS_SUBMISSION_NEW/data/targets/till_end_mimic_iv_extra_features_train_NOTE_TARGET_2_rad_all.csv'

train_emb_seq = pd.read_csv(train_emb_seq_path, converters={'emb_seq': convert_to_nested_list})
train_emb = torch.load(f'{"/home/ugrads/a/aa_ron_su/BoXHED_Fuse/BoXHED_Fuse/JSS_SUBMISSION_NEW/data/embs/Clinical-T5-Base_rad_all_out/from_epoch6/3"}/train_embs.pt')
train_target = pd.read_csv(train_target_path, converters = {'NOTE_ID_SEQ': convert_to_list})



In [4]:
print('merging data into df with cols {train_embs_seq, label}')
target = 'delta_in_2_days' 
train_target.rename(columns = {target:'label'}, inplace=True)
train_target_exploded = explode_train_target(train_target)
train_emb_df = pd.DataFrame()
train_emb_df['emb'] = [np.array(e) for e in train_emb]
train_emb_df = pd.concat([train_target_exploded, train_emb_df], axis=1)
# print('merging on NOTE_ID_SEQ...')
# train_emb_seq = merge_embs_to_seq(train_target, train_embs=train_emb_df)
# validate_train_emb_seq(train_emb_seq, train_target)
# train_emb_seq.to_csv(train_emb_seq_path, index=False)

merging data into df with cols {train_embs_seq, label}


In [5]:
train_target_exploded.reset_index(inplace=True)

In [6]:
indexes = [np.random.randint(0, 76158) for _ in range(36)]  # Generates a random integer in the range [0, 293060]]
labels = train_target.label
list_IDs = train_target.NOTE_ID_SEQ
exploded_idxs = []
embs = []

import time
t0 = time.time()
for _ in range(10):
    for i in indexes:
        y = labels[i]
        note_id_seq = list_IDs[i]
        # note_id_seq_df =  pd.DataFrame(note_id_seq, columns=['NOTE_ID'])
        exploded_idx = pd.merge(pd.DataFrame({'NOTE_ID': note_id_seq}, columns=['NOTE_ID']),
                        train_target_exploded,
                        how='left',
                        on=['NOTE_ID'])
        exploded_idxs.append(exploded_idx)
        emb = train_emb[exploded_idx.index]
        embs.append(emb)
            
            
            
            # note_id_seq_df =  pd.DataFrame(note_id_seq, columns=['NOTE_ID'])
            # train_embs_seq = pd.merge(note_id_seq_df,
            #             train_emb_df,
            #             how = 'left',
            #             on = ['NOTE_ID'])
    print(f'time: {time.time() - t0}')
    # doc_seqs = torch.cat(list(self.df[self.df.HADM_ID == index].DOC_EMB.values), 0)

    

time: 5.555430173873901
time: 10.739282369613647
time: 16.29861831665039
time: 21.99062490463257
time: 27.184352159500122
time: 32.60404181480408
time: 38.10696363449097
time: 43.36784505844116
time: 48.41510725021362
time: 53.436328649520874


In [93]:
train_target

,ICUSTAY_ID,NOTE_ID,label,NOTE_ID_SEQ
0,30000484.0,18421337-RR-50,0,"[18421337-RR-6, 18421337-RR-7, 18421337-RR-10,..."
1,30001148.0,12980335-RR-16,1,"[12980335-RR-15, 12980335-RR-16]"
2,30001446.0,16513856-RR-21,0,"[16513856-RR-12, 16513856-RR-13, 16513856-RR-1..."
3,30001446.0,16513856-RR-23,0,"[16513856-RR-12, 16513856-RR-13, 16513856-RR-1..."
4,30001446.0,16513856-RR-22,0,"[16513856-RR-12, 16513856-RR-13, 16513856-RR-1..."
...,...,...,...,...
76153,39999286.0,11305179-RR-30,0,"[11305179-RR-10, 11305179-RR-13, 11305179-RR-1..."
76154,39999301.0,16180713-RR-9,1,"[16180713-RR-5, 16180713-RR-6, 16180713-RR-7, ..."
76155,39999552.0,11256534-RR-5,1,"[11256534-RR-3, 11256534-RR-4, 11256534-RR-5]"
76156,39999552.0,11256534-RR-6,0,"[11256534-RR-3, 11256534-RR-4, 11256534-RR-5, ..."


In [89]:
indexes.__len__()


36

,ICUSTAY_ID,NOTE_ID,label,NOTE_ID_SEQ
0,30000484.0,18421337-RR-50,0,"[18421337-RR-6, 18421337-RR-7, 18421337-RR-10,..."
1,30001148.0,12980335-RR-16,1,"[12980335-RR-15, 12980335-RR-16]"
2,30001446.0,16513856-RR-21,0,"[16513856-RR-12, 16513856-RR-13, 16513856-RR-1..."
3,30001446.0,16513856-RR-23,0,"[16513856-RR-12, 16513856-RR-13, 16513856-RR-1..."
4,30001446.0,16513856-RR-22,0,"[16513856-RR-12, 16513856-RR-13, 16513856-RR-1..."
...,...,...,...,...
76153,39999286.0,11305179-RR-30,0,"[11305179-RR-10, 11305179-RR-13, 11305179-RR-1..."
76154,39999301.0,16180713-RR-9,1,"[16180713-RR-5, 16180713-RR-6, 16180713-RR-7, ..."
76155,39999552.0,11256534-RR-5,1,"[11256534-RR-3, 11256534-RR-4, 11256534-RR-5]"
76156,39999552.0,11256534-RR-6,0,"[11256534-RR-3, 11256534-RR-4, 11256534-RR-5, ..."


In [52]:
train_target_exploded

,ICUSTAY_ID,NOTE_ID_RECENT,NOTE_ID,label
0,30000484.0,18421337-RR-50,18421337-RR-6,0
1,30000484.0,18421337-RR-50,18421337-RR-7,0
2,30000484.0,18421337-RR-50,18421337-RR-10,0
3,30000484.0,18421337-RR-50,18421337-RR-8,0
4,30000484.0,18421337-RR-50,18421337-RR-9,0
...,...,...,...,...
293055,39999552.0,11256534-RR-5,11256534-RR-3,0
293056,39999552.0,11256534-RR-5,11256534-RR-4,0
293057,39999552.0,11256534-RR-5,11256534-RR-5,1
293058,39999552.0,11256534-RR-6,11256534-RR-6,0


In [57]:
train_emb.shape

torch.Size([293060, 64])

In [48]:
train_embs_seq.emb

0    [-0.9415110780984006, -0.9966442662679738, 0.9...
1    [-0.8654147320943966, -0.9872478068220216, 0.9...
2    [-0.9242474677569906, -0.9945977736198899, 0.9...
3    [-0.9444604269259617, -0.9962892972857893, 0.9...
Name: emb, dtype: object